<a href="https://colab.research.google.com/github/OzanGenc/TurkishCuisineDataAnalysis/blob/main/StreamTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tweepy import OAuthHandler
from tweepy import API 
from tweepy import Stream
import pickle
import json
from tweepy.streaming import StreamListener
import time
import sys

In [ ]:
class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        #self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
        
        self.jsonlist = []


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    

    def on_status(self, status):
        
        self.jsonlist.append(str(status))
        self.counter += 1
        
        if self.counter >= 2:
            pickle.dump( self.jsonlist, open( "tweets_streamed.p", "wb" ) )
            #self.output.write(self.jsonlist)
            #self.output.close()
            
            raise Exception("Sorry, streaming limit exceeded")
            
        
            #self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            #self.counter = 0
        return


    '''    
        
    def on_status(self, status):
        self.output.write(status)
        self.output.write(',')
        self.counter += 1
        if self.counter >= 2:
            self.output.close()
            return False
            #self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            #self.counter = 0
        return

    '''
    
    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):

        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

In [ ]:
key_dict = pickle.load( open( "twitter_keys.p", "rb" ) )

access_token = key_dict['access_token']
access_token_secret = key_dict['access_token_secret']
consumer_key = key_dict['consumer_key']
consumer_secret = key_dict['consumer_secret']

In [ ]:
# Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)

# Access key authentication
auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth)

In [ ]:
# Set up words to track
keywords_to_track = ['yemek']

In [ ]:
# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data

#stream.sample()
stream.filter(track = keywords_to_track, languages=["tr"])

Exception: ignored

In [ ]:
tweets_list = pickle.load( open( "tweets_streamed.p", "rb" ) )

In [ ]:
tweet_json = tweets_list[0]

In [ ]:
# Convert from JSON to Python object
tweet = json.loads(tweet_json)

# Print tweet text
print(tweet['retweeted_status']['extended_tweet']['full_text'])

# Print tweet id
print(tweet['id'])

KeyError: ignored

In [ ]:
tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Dec 02 07:48:10 +0000 2020',
 'display_text_range': [10, 64],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 2955315479,
    'id_str': '2955315479',
    'indices': [0, 9],
    'name': 'BPT',
    'screen_name': 'bpthaber'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1334041653120815104,
 'id_str': '1334041653120815104',
 'in_reply_to_screen_name': 'bpthaber',
 'in_reply_to_status_id': 1333768550016757760,
 'in_reply_to_status_id_str': '1333768550016757760',
 'in_reply_to_user_id': 2955315479,
 'in_reply_to_user_id_str': '2955315479',
 'is_quote_status': False,
 'lang': 'tr',
 'place': None,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': '@bpthaber Araba bilerek yayaların üstüne sürdüğü için kaza olmu

In [ ]:
import sys

from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener

# Replace the "None"s by your own credentials
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = output_file
    def on_status(self, status):
        print(status.text, file=self.output_file)
    def on_error(self, status_code):
        print(status_code)
        return False

output = open('stream_output.json', 'w')
listener = Listener(output_file=output)
keywords_to_track = ['#rstats', '#python']

stream = Stream(auth=api.auth, listener=listener)
try:
    print('Start streaming.')
    
    stream.filter(track = keywords_to_track)
    #stream.sample(languages=['en'])
    
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()
    output.close()

In [ ]:
tweet_json = 'stream_output.json'

In [ ]:
# Convert from JSON to Python object
tweet = json.loads(tweet_json)

# Print tweet text
print(tweet['text'])

# Print tweet id
print(tweet['id'])

In [ ]:
#limiting the streamed data from twitter


class listener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)